<a href="https://colab.research.google.com/github/arutraj/ML_Basics/blob/main/13_17_Model_Checkpointing_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps to solve Emergency vs Non-Emergency vehicle classification

<ol>1. Loading the dataset</ol>
<ol>2. Pre-processing the data</ol>
<ol>3. Creating training and validation set</ol>
<ol>4. Defining the model architecture</ol>
<ol>5. Compiling the model</ol>
<ol>6. Training the model</ol>
<ol><ol>Setting up model checkpointing</ol></ol>
<ol>7. Evaluating model performance</ol>

## 1. Loading the dataset

In [3]:
# import necessary libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# importing layers from keras
from keras.layers import Dense, InputLayer
from keras.models import Sequential
# importing adam optimizer from keras optimizer module
from tensorflow.keras.optimizers import Adam

# train_test_split to create training and validation set
from sklearn.model_selection import train_test_split
# accuracy_score to calculate the accuracy of predictions
from sklearn.metrics import accuracy_score

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# unzipping the Dataset.zip file
!ls /content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/*jpg |wc -l

2352


In [4]:
# reading the csv file
data = pd.read_csv('/content/emergency_classification.csv')

In [5]:
# defining the seed value
seed = 42

In [6]:
# looking at first five rows of the data
data.head()

,image_names,emergency_or_not
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,1
4,4.jpg,1


In [7]:
# load images and store it in numpy array

# empty list to store the images
X = []
# iterating over each image
for img_name in data.image_names:
    # loading the image using its name
    img = plt.imread('/content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/' + img_name)
    # saving each image in the list
    X.append(img)

# converting the list of images into array
X=np.array(X)

# storing the target variable in separate variable
y = data.emergency_or_not.values

In [8]:
# shape of the images
X.shape

(2352, 224, 224, 3)

## 2. Pre-processing the data

In [9]:
# converting 3 dimensional image to 1 dimensional image
X = X.reshape(X.shape[0], 224*224*3)
X.shape

(2352, 150528)

In [10]:
# minimum and maximum pixel values of images
X.min(), X.max()

(0, 255)

In [11]:
# normalizing the pixel values
X = X / X.max()

In [12]:
# minimum and maximum pixel values of images after normalizing
X.min(), X.max()

(0.0, 1.0)

## 3. Creating training and validation set

In [13]:
# creating a training and validation set
X_train, X_valid, y_train, y_valid=train_test_split(X,y,test_size=0.3, random_state=seed)

In [14]:
# shape of training and validation set
(X_train.shape, y_train.shape), (X_valid.shape, y_valid.shape)

(((1646, 150528), (1646,)), ((706, 150528), (706,)))

## 4. Defining the model architecture

In [15]:
# defining the model architecture
model=Sequential()

model.add(InputLayer(input_shape=(224*224*3,)))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

## 5. Compiling the model

In [17]:
# defining the adam optimizer and setting the learning rate as 10^-5
adam = Adam(learning_rate=1e-5)

In [18]:
# compiling the model

# defining loss as binary crossentropy
# defining optimizer as Adam
# defining metrics as accuracy

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

## 6. Training the model

### Setting up model checkpointing

In [19]:
# importing model checkpointing from keras callbacks
from keras.callbacks import ModelCheckpoint

In [20]:
# defining model checkpointing

# defining the path to store the weights
filepath="best_weights.hdf5"

# defining the model checkpointing and metric to monitor
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# defining checkpointing variable
callbacks_list = [checkpoint]

In [21]:
# training the model for 50 epochs
model_history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_valid,y_valid), callbacks=callbacks_list)

Epoch 1/50
12/13 [==========================>...] - ETA: 0s - loss: 0.6665 - accuracy: 0.6022
Epoch 1: val_accuracy improved from -inf to 0.61048, saving model to best_weights.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13/13 [==============================] - 9s 428ms/step - loss: 0.6667 - accuracy: 0.6015 - val_loss: 0.6494 - val_accuracy: 0.6105
Epoch 2/50
13/13 [==============================] - ETA: 0s - loss: 0.6384 - accuracy: 0.6488
Epoch 2: val_accuracy improved from 0.61048 to 0.63456, saving model to best_weights.hdf5
13/13 [==============================] - 4s 302ms/step - loss: 0.6384 - accuracy: 0.6488 - val_loss: 0.6357 - val_accuracy: 0.6346
Epoch 3/50
13/13 [==============================] - ETA: 0s - loss: 0.6237 - accuracy: 0.6598
Epoch 3: val_accuracy improved from 0.63456 to 0.64589, saving model to best_weights.hdf5
13/13 [==============================] - 8s 634ms/step - loss: 0.6237 - accuracy: 0.6598 - val_loss: 0.6273 - val_accuracy: 0.6459
Epoch 4/50
13/13 [==============================] - ETA: 0s - loss: 0.6151 - accuracy: 0.6683
Epoch 4: val_accuracy improved from 0.64589 to 0.66431, saving model to best_weights.hdf5
13/13 [==============================] - 8s 680ms/step 

## 8. Evaluating model performance

In [23]:
prediction = model.predict(X_valid)
prediction = prediction.reshape(706,)

# converting probabilities to classes
prediction_int = prediction >= 0.5
prediction_int = prediction_int.astype(int)

23/23 [==============================] - 0s 11ms/step


In [24]:
# accuracy on validation set
print('Accuracy on validation set:', accuracy_score(y_valid, prediction_int), '%')

Accuracy on validation set: 0.7195467422096318 %


In [27]:
# loading the best model
model.load_weights("best_weights.hdf5")
#data.Mileage.isnull().sum()

In [29]:
prediction = model.predict(X_valid)
prediction = prediction.reshape(706,)

# converting probabilities to classes
prediction_int = prediction >= 0.5
prediction_int = prediction_int.astype(int)

23/23 [==============================] - 0s 16ms/step


In [ ]:
# accuracy on validation set
print('Accuracy on validation set:', accuracy_score(y_valid, prediction_int), '%')

Accuracy on validation set: 0.7209631728045326 %
